# Dremio Test

## Setting up dremio


## Imports

In [1]:
import os
import sys
import atoti as tt

In [2]:
import pandas as pd

Make sure you are running a version of atoti older than 0.8.10. This has been tested on 0.8.5

In [3]:
tt._version.VERSION

'0.9.0.4972.20240510095356'

### Jar Input

In [4]:
dremio_driver_jar="flight-sql-jdbc-driver-15.0.0.jar"
dremio_dialect_jar="dremio-dialect-0.2.0.jar"

### SQL Setup

In [5]:
schema_name='"@admin"'
db_table_name='SampleData20240104'
sql = f"SELECT * FROM {schema_name}.{db_table_name}"

### Database setup

In [6]:
username="admin"
password="admin01!"
host_name="localhost"
port="32010"
database = "Dremio"

jdbc_legacy_url=f"jdbc:dremio:direct={host_name}:31010?user={username}&password={password}&useEncryption=false"
jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&password={password}&useEncryption=false"

jdbc_driver="org.apache.arrow.driver.jdbc.ArrowFlightJdbcDriver"
jdbc_legacy_driver="com.dremio.jdbc.Driver"
atoti_table_name="table_facts"

In [7]:
os.getcwd()

'/Users/gpadya/Documents/dremio-driver/Python Dremio Demo'

## Session

In [8]:
session = tt.Session(
    port=9092,
    user_content_storage = "./Python Dremio Demo/content",
    java_options=[
        "-Xmx10g",
        "-XX:MaxDirectMemorySize=20g",
        # "-Dorg.springframework.boot.logging.LoggingSystem=none"
    ],
    extra_jars=[f"./utilities/extra_jars/{dremio_driver_jar}",
               f"./utilities/extra_jars/{dremio_dialect_jar}"],
    logging=tt.LoggingConfig(destination="./logging/server.log")
)

# Configuration of the Data model

## Connect to Dremio and Create External DataStore

In [9]:
from atoti_directquery_jdbc import JDBCConnectionInfo

In [10]:
from atoti_directquery_jdbc import (
JDBCConnection, 
JDBCConnectionInfo,
JDBCTable,
JDBCTableOptions
)

In [11]:
jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&password={password}&useEncryption=false"

In [12]:
# jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&useEncryption=false"

In [13]:
connection_info=JDBCConnectionInfo(
    jdbc_url,
    sql_dialect_key="Dremio",
    # password=password,
    password_parameter_name='password'
    
)
external_database = session.connect_to_external_database(connection_info)

In [14]:
external_database.tables

## Set up DQ Tables

In [15]:
table_facts=session.add_external_table(external_database.tables['vw_SampleData20240104'])

In [16]:
# table_facts_types= {
#     "AsofDate" : tt.LOCAL_DATE,
#     "CUSIP" : tt.STRING,
#     "SectorCode" : tt.STRING,
#     "Strategy" : tt.STRING,
#     "MarketValue" : tt.DOUBLE,
#     "EffectiveDuration" : tt.DOUBLE,
#     "OptionAdjustedSpread" : tt.DOUBLE,
#     "LegalEntity" : tt.STRING,
#     "PortfolioTicker" : tt.STRING,
#     "OwnershipType" : tt.STRING,
#     "PercentWeight" : tt.INT
# }

In [17]:
# table_facts=session.add_external_table(table_facts_db)
table_facts.head(10)

,AsofDate,CUSIP,SectorCode,Strategy,MarketValue,EffectiveDuration,OptionAdjustedSpread,LegalEntity,PortfolioTicker,OwnershipType,PercentWeight
0,2024-01-04,91282CFV8,Peaches,Paris,17042127.4,8.809704,3.81681,GEA_BM,NEABMLCWF,Passive,1.0
1,2024-01-04,91282CFV8,Peaches,Paris,17042127.4,8.809704,3.81681,GEA_BM,NEABMLCWF,Active,1.0
2,2024-01-04,91282CEV9,Peaches,Paris,6743085.93,5.95656,0.0,GEA_BM,NEABMLCWF,Passive,1.0
3,2024-01-04,91282CEV9,Peaches,Paris,6743085.93,5.95656,0.0,GEA_BM,NEABMLCWF,Active,1.0
4,2024-01-04,91282CFT3,Peaches,Paris,12553750.93,6.138696,0.12888,GEA_BM,NEABMLCWF,Passive,1.0
5,2024-01-04,91282CFT3,Peaches,Paris,12553750.93,6.138696,0.12888,GEA_BM,NEABMLCWF,Active,1.0
6,2024-01-04,ZXW2J9PF8,Pineapples,Paris,2287540.51,2.80962,36.01908,GEA_BM,NEABMLCWF,Passive,1.0
7,2024-01-04,ZXW2J9PF8,Pineapples,Paris,2287540.51,2.80962,36.01908,GEA_BM,NEABMLCWF,Active,1.0
8,2024-01-04,91282CGC9,Peaches,Paris,7935307.75,4.391112,0.0,GEA_BM,NEABMLCWF,Passive,1.0
9,2024-01-04,91282CGC9,Peaches,Paris,7935307.75,4.391112,0.0,GEA_BM,NEABMLCWF,Active,1.0


In [18]:
session.tables.schema

```mermaid
erDiagram
  "vw_SampleData20240104" {
    _ LocalDate "AsofDate"
    _ String "CUSIP"
    _ String "SectorCode"
    _ String "Strategy"
    nullable double "MarketValue"
    nullable double "EffectiveDuration"
    nullable double "OptionAdjustedSpread"
    _ String "LegalEntity"
    _ String "PortfolioTicker"
    _ String "OwnershipType"
    nullable double "PercentWeight"
  }
```


## Create Cube

In [19]:
cube = session.create_cube(table_facts, name="facts", mode="no_measures")
h, l, m = cube.hierarchies, cube.levels, cube.measures

### Clean Up Hierarchies

In [20]:
# del(h['MarketValue'],h['EffectiveDuration'], h['OptionAdjustedSpread'], h['PercentWeight'])

## Add Measures

In [21]:
%%time
m["MarketValue"] = tt.agg.sum(table_facts["MarketValue"])
m["EffectiveDuration"] = tt.agg.sum(table_facts["EffectiveDuration"])
m["EffectiveDuration"] = tt.agg.sum(table_facts["EffectiveDuration"])
m["OptionAdjustedSpread"] = tt.agg.sum(table_facts["OptionAdjustedSpread"])
m["PercentWeight"] = tt.agg.sum(table_facts["PercentWeight"])

CPU times: user 3.16 ms, sys: 4.3 ms, total: 7.46 ms
Wall time: 96.2 ms


In [22]:
%%time
m["MV Duration"] = tt.agg.sum_product(
    table_facts["MarketValue"],table_facts["EffectiveDuration"],table_facts["PercentWeight"])

CPU times: user 4.82 ms, sys: 6.82 ms, total: 11.6 ms
Wall time: 102 ms


In [23]:
%%time
m["MV PercentWeight"] = tt.agg.sum_product(table_facts["MarketValue"],table_facts["PercentWeight"])

CPU times: user 2.55 ms, sys: 3.23 ms, total: 5.78 ms
Wall time: 30.7 ms


In [24]:
%%time
m["MV OAS"] = tt.agg.sum_product(table_facts["MarketValue"],table_facts["OptionAdjustedSpread"],table_facts["PercentWeight"])

CPU times: user 3.27 ms, sys: 3.76 ms, total: 7.04 ms
Wall time: 44.3 ms


In [25]:
%%time
m["Weighted Average Duration"] = tt.where(m["MV PercentWeight"] == 0, 0, m["MV Duration"]/m["MV PercentWeight"])

CPU times: user 7.93 ms, sys: 8.19 ms, total: 16.1 ms
Wall time: 108 ms


In [26]:
%%time
m["Weighted Average Duration OAS"] = tt.where(m["MV PercentWeight"] == 0, 0, m["MV OAS"]/m["MV PercentWeight"])

CPU times: user 2.72 ms, sys: 2.87 ms, total: 5.59 ms
Wall time: 21.4 ms


## Link to UI

In [27]:
session.link

http://localhost:9092

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.